In [2]:
import os
import warnings

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
import seaborn as sns

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

Data source: Colorado Information Marketplace https://data.colorado.gov/Nonprofit-Data/Charity-Filed-Financial-Information-in-Colorado/37wu-kn3g

In [5]:
char_tmp = pd.read_csv('../../npsg_datafiles/Charity_Filed_Financial_Information_in_Colorado.csv', encoding = 'latin-1') 

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6,8,9,21,84,85,89,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
char_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59097 entries, 0 to 59096
Data columns (total 93 columns):
entityId                                                    59097 non-null int64
registrantTypeAbbr                                          59097 non-null object
name                                                        59097 non-null object
fein                                                        58232 non-null float64
filingType                                                  59097 non-null object
documentId                                                  59089 non-null float64
amendmentEntityId                                           52345 non-null object
externalFilingFromNccs                                      59097 non-null bool
externalFilingType                                          4 non-null object
actualFinancialsProvided                                    59033 non-null object
principalAddress                                            59096 non-null o

#### Counts regardless of whether the principal state is Colorado

In [10]:
char_tmp['entityId'].groupby(char_tmp['nteeCode1']).count()

nteeCode1
ANIMALS                                                3559
ARTS,CULTURE & HUMANITIES                              7612
CIVIL RIGHTS,SOCIAL ACTION,& ADVOCACY                  3788
COMMUNITY IMPROVEMENT,CAPACITY BUILDING                4244
CRIME,LEGAL                                             452
DISEASES,DISORDERS,& MEDICAL DISCIPLINES               2756
EDUCATION                                             12795
EMPLOYMENT                                              244
ENVIRONMENT QUALITY,PROTECTION & BEAUTIFICATION        1498
FOOD,AGRICULTURE,& NUTRITION                            909
HEALTH                                                 3080
HOUSING,SHELTER                                        1375
HUMAN SERVICES                                         3568
INTERNATIONAL,FOREIGN AFFAIRS, & NATIONAL SECURITY      341
MEDICAL RESEARCH                                        287
MENTAL HEALTH,CRISIS INTERVENTION                       298
MUTUAL,MEMBERSHIP BENEFIT     

#### Should we restrict charities to those whose principalState is Colorado? Could also try to use organizationEstablishedState and incorporationState

In [11]:
is_colorado = char_tmp['principalState'] == 'CO'

In [12]:
char_tmp_co = char_tmp[is_colorado]

#### Summarize by activity code where principalState is Colorado

In [13]:
char_tmp_co['entityId'].groupby(char_tmp_co['nteeCode1']).count()

nteeCode1
ANIMALS                                               2219
ARTS,CULTURE & HUMANITIES                             6473
CIVIL RIGHTS,SOCIAL ACTION,& ADVOCACY                 1807
COMMUNITY IMPROVEMENT,CAPACITY BUILDING               3224
CRIME,LEGAL                                            249
DISEASES,DISORDERS,& MEDICAL DISCIPLINES              1007
EDUCATION                                             8636
EMPLOYMENT                                             164
ENVIRONMENT QUALITY,PROTECTION & BEAUTIFICATION        952
FOOD,AGRICULTURE,& NUTRITION                           613
HEALTH                                                1790
HOUSING,SHELTER                                       1042
HUMAN SERVICES                                        2038
INTERNATIONAL,FOREIGN AFFAIRS, & NATIONAL SECURITY      64
MEDICAL RESEARCH                                        31
MENTAL HEALTH,CRISIS INTERVENTION                      220
MUTUAL,MEMBERSHIP BENEFIT                     

#### Looks like there is a little cleanup to be done on residentialCounty. Also, need a definition of residentialCounty. 

In [14]:
char_tmp_co['entityId'].groupby(char_tmp_co['residentialCounty']).count()

residentialCounty
ADAMS                       926
ALAMOSA                     415
ARAPAHOE                   2149
ARCHULETA                   395
BACA                          7
BENT                         25
BEXAR                         5
BOULDER                    2895
BROOMFIELD                  251
CALIFORNIA                    2
CHAFFEE                     352
CHEYENNE                      2
CLEAR CREEK                 195
CO                          137
CO - COLORADO                17
COLORADO                    211
COLORADO (CO)                24
COLORADO [CO]                 3
CONEJOS                      28
CONTRA COSTA                  3
COSTILLA                     81
CROWLEY                       1
CUSTER                      232
DELTA                       475
DENVER                     4096
DOLORES                      37
DOUGLAS                    1137
EAGLE                      1075
EL PASO                    3493
ELBERT                       88
FREDERICKSBURG (INDEP.

#### Look at spending on program services. Could compute per capita figure after acquiring county population

In [15]:
char_tmp_co['expensesFromProgramServices'].groupby(char_tmp_co['residentialCounty']).describe()

,count,mean,std,min,25%,50%,75%,max
residentialCounty,,,,,,,,
ADAMS,909.0,3.121984e+06,5.580829e+07,-5.962280e+08,8.149240e+03,4.433600e+04,1.443710e+05,9.418800e+08
ALAMOSA,412.0,4.984057e+05,2.574445e+06,0.000000e+00,1.727142e+04,1.161140e+05,3.575539e+05,2.716011e+07
ARAPAHOE,2129.0,4.002309e+05,1.356761e+06,0.000000e+00,8.261180e+03,4.965000e+04,2.536380e+05,2.510490e+07
ARCHULETA,393.0,1.218708e+05,1.876962e+05,-2.639200e+04,8.989000e+03,4.000000e+04,1.633380e+05,1.225337e+06
BACA,7.0,7.150404e+03,9.890419e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.252642e+04,2.500000e+04
BENT,25.0,4.024324e+04,4.425821e+04,0.000000e+00,0.000000e+00,4.370900e+04,7.038700e+04,1.723290e+05
BEXAR,5.0,1.871600e+05,1.348344e+05,6.250000e+02,1.181570e+05,2.255870e+05,2.320000e+05,3.594310e+05
BOULDER,2887.0,2.704821e+06,2.162949e+07,0.000000e+00,1.706200e+04,6.816600e+04,2.485915e+05,2.725926e+08
BROOMFIELD,250.0,1.814930e+05,3.202370e+05,0.000000e+00,2.450775e+04,6.956250e+04,1.946798e+05,2.190045e+06
